<a href="https://colab.research.google.com/github/anicelysantos/book-python-para-analise-de-dados/blob/main/agregacao_operacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Funcionamento do GroupBy

In [2]:
df = pd.DataFrame({'key1':['a','a','b','b','a'], 'key':['one','two','one','two','one'], 'data1': np.random.randn(5), 'data2': np.random.randn(5)})
df

,key1,key,data1,data2
0,a,one,-0.087628,-0.781517
1,a,two,-1.033337,-1.538188
2,b,one,-1.680211,-0.884163
3,b,two,-1.965759,-0.567086
4,a,one,-0.328473,-0.429713


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a   -0.483146
b   -1.822985
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key']]).mean()
means

key1  key
a     one   -0.208050
      two   -1.033337
b     one   -1.680211
      two   -1.965759
Name: data1, dtype: float64

In [6]:
means.unstack()

key,one,two
key1,,
a,-0.208050,-1.033337
b,-1.680211,-1.965759


In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -1.033337
            2006   -1.680211
Ohio        2005   -1.026693
            2006   -0.328473
Name: data1, dtype: float64

In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.483146,-0.916473
b,-1.822985,-0.725624


In [9]:
df.groupby(['key1', 'key']).mean()

data1     data2
key1 key                    
a    one -0.208050 -0.605615
     two -1.033337 -1.538188
b    one -1.680211 -0.884163
     two -1.965759 -0.567086

In [10]:
df.groupby(['key1', 'key']).size()

key1  key
a     one    2
      two    1
b     one    1
      two    1
dtype: int64

**Iterando por grupos**

In [11]:
for nome, grupo in df.groupby('key1'):
  print(nome)
  print(grupo)

a
  key1  key     data1     data2
0    a  one -0.087628 -0.781517
1    a  two -1.033337 -1.538188
4    a  one -0.328473 -0.429713
b
  key1  key     data1     data2
2    b  one -1.680211 -0.884163
3    b  two -1.965759 -0.567086


In [12]:
for (k1,k2), grupo in df.groupby(['key1','key']):
  print((k1,k2))
  print(grupo)

('a', 'one')
  key1  key     data1     data2
0    a  one -0.087628 -0.781517
4    a  one -0.328473 -0.429713
('a', 'two')
  key1  key     data1     data2
1    a  two -1.033337 -1.538188
('b', 'one')
  key1  key     data1     data2
2    b  one -1.680211 -0.884163
('b', 'two')
  key1  key     data1     data2
3    b  two -1.965759 -0.567086


In [13]:
#gerando dicionario com uma parte dos dados

parte = dict(list(df.groupby('key1')))
parte['b']

,key1,key,data1,data2
2,b,one,-1.680211,-0.884163
3,b,two,-1.965759,-0.567086


In [14]:
df.dtypes

key1      object
key       object
data1    float64
data2    float64
dtype: object

In [15]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, grupo in grouped:
  print(dtype)
  print(grupo)

float64
      data1     data2
0 -0.087628 -0.781517
1 -1.033337 -1.538188
2 -1.680211 -0.884163
3 -1.965759 -0.567086
4 -0.328473 -0.429713
object
  key1  key
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


**Selecionando uma coluna ou um subconjunto de colunas**

In [16]:
df.groupby(['key1', 'key'])[['data2']].mean()

data2
key1 key          
a    one -0.605615
     two -1.538188
b    one -0.884163
     two -0.567086

In [17]:
s_grouped = df.groupby(['key1', 'key'])['data2']
s_grouped

In [18]:
s_grouped.mean()

key1  key
a     one   -0.605615
      two   -1.538188
b     one   -0.884163
      two   -0.567086
Name: data2, dtype: float64

**Agrupando com dicionário e séries**

In [19]:
people = pd.DataFrame(np.random.randn(5,5), columns=['a','b','c','d','e'], index=['Joe', 'Steve','Wes','Jim','Travis'])

In [20]:
people.iloc[2:3, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,-0.938876,0.291435,1.262568,0.360246,-0.875104
Steve,0.646470,0.515857,-0.843998,1.431758,-0.717924
Wes,-1.114920,NaN,NaN,-1.846784,0.113512
Jim,-0.271455,-0.350433,-1.612754,0.308810,1.113253
Travis,0.253249,0.810306,-1.319260,-1.248249,1.589779


In [21]:
#somar as colunas por grupo
mapping = {'a': 'red', 'b':'red', 'c':'blue', 'd':'blue','e':'red','f':'orange'}

In [22]:
by_columns = people.groupby(mapping, axis=1)
by_columns.sum()

,blue,red
Joe,1.622813,-1.522545
Steve,0.587759,0.444403
Wes,-1.846784,-1.001408
Jim,-1.303944,0.491365
Travis,-2.567509,2.653334


In [23]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [24]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


**Agrupando com funções**

In [25]:
#agrupando por tamanho de string (contando string do índice)
people.groupby(len).sum()

,a,b,c,d,e
3,-2.325251,-0.058998,-0.350186,-1.177729,0.351662
5,0.646470,0.515857,-0.843998,1.431758,-0.717924
6,0.253249,0.810306,-1.319260,-1.248249,1.589779


In [26]:
key_list = ['one', 'one', 'one','two','two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.114920  0.291435  1.262568 -1.846784 -0.875104
  two -0.271455 -0.350433 -1.612754  0.308810  1.113253
5 one  0.646470  0.515857 -0.843998  1.431758 -0.717924
6 two  0.253249  0.810306 -1.319260 -1.248249  1.589779

**Agrupando por níveis de índice**

In [27]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'], [1,3,5,1,3]], names=['cty','tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      2.812185 -0.076178  0.530321  0.798239 -0.775088
1      1.221232  0.494571  0.377136 -0.240049  0.396134
2      0.194545  0.935806  0.350637  0.627208 -1.401658
3      0.152571  0.676929 -0.575866 -0.126042  0.808398

In [28]:
hier_df.groupby(level = 'cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# Agregação de dados

In [29]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a   -0.135797
b   -1.708766
Name: data1, dtype: float64

In [30]:
#Para usar uma função própria, utilize agg e passe a função entre parenteses
def peak_to_peak(arr):
  return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.945709,1.108475
b,0.285548,0.317078


In [31]:
grouped.describe()

data1                      ...     data2                    
     count      mean       std  ...       50%       75%       max
key1                            ...                              
a      3.0 -0.483146  0.491461  ... -0.781517 -0.605615 -0.429713
b      2.0 -1.822985  0.201913  ... -0.725624 -0.646355 -0.567086

[2 rows x 16 columns]

**Aplicação de função nas colunas e aplicação de várias funções**

In [32]:
tips = pd.read_csv('/content/drive/MyDrive/dados_pandas/pydata-book-2nd-edition/examples/tips.csv')

In [33]:
#Acrescenta a porcentagem de gorjeta sobre o total da conta
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [34]:
grouped = tips.groupby(['day','smoker'])
grouped_pct = grouped['tip_pct']
#É possivel passar uma função descritiva como string
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [35]:
grouped_pct.agg(['mean','std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [36]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [37]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [38]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [39]:
#personalizar o nome das colunas
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [40]:
grouped.agg({'tip': np.max, 'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [41]:
grouped.agg({'tip_pct':['min','max','mean','std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

**Devolvendo dados agregados sem índice de linha**

In [42]:
tips.groupby(['day','smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# Método apply: separar-aplicar-combinar genérico

Usando a base de dados de gorjetas vamos selecionar os cinco primeiros valores de `tip_pct`.

In [43]:
#Essa função seleciona as linhas com os maiores valores em uma coluna
def top (df, n=5, column = 'tip_pct'):
  return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [44]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [45]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [46]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [47]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [48]:
#mesma coisa da célula anterior
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

**Suprimindo as chaves de grupo**

In [49]:
#ocultar os índices hierarquicos
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


**Análise de quantis e de buckets**

In [50]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2':np.random.randn(1000)})

In [51]:
quartiles = pd.cut(frame.data1, 4)

In [52]:
quartiles[:10]

0     (0.0286, 1.841]
1    (-1.783, 0.0286]
2     (0.0286, 1.841]
3    (-1.783, 0.0286]
4    (-1.783, 0.0286]
5     (0.0286, 1.841]
6    (-1.783, 0.0286]
7     (0.0286, 1.841]
8    (-1.783, 0.0286]
9     (0.0286, 1.841]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.603, -1.783] < (-1.783, 0.0286] < (0.0286, 1.841] <
                                    (1.841, 3.653]]

In [53]:
def get_stats(group):
  return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean':group.mean()}

In [54]:
grouped = frame.data2.groupby(quartiles)

In [55]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.603, -1.783]",-2.171576,1.757315,55.0,-0.060771
"(-1.783, 0.0286]",-3.504567,2.517869,473.0,0.047023
"(0.0286, 1.841]",-2.848933,2.621860,449.0,-0.028949
"(1.841, 3.653]",-1.964970,1.350980,23.0,0.066947


In [56]:
#Obter só o número dos quantis

grouping = pd.qcut(frame.data1, 10, labels = False)

In [57]:
grouped = frame.data2.groupby(grouping)

In [58]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.171576,2.178041,100.0,-0.039485
1,-2.026593,2.517869,100.0,0.111008
2,-2.069152,2.248274,100.0,0.198256
3,-3.504567,2.141294,100.0,-0.010792
4,-3.221085,2.355536,100.0,-0.055707
5,-2.848933,2.141268,100.0,-0.038064
6,-2.366739,1.891016,100.0,-0.070531
7,-2.335437,2.152606,100.0,-0.033606
8,-2.683618,2.621860,100.0,-0.002978


**Preenchendo valores ausentes com valores específicos de grupo**

In [59]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.944933
2         NaN
3    0.906374
4         NaN
5   -0.633858
dtype: float64

In [60]:
#preenchendo os valores ausentes com a media dos números
s.fillna(s.mean())

0   -0.224139
1   -0.944933
2   -0.224139
3    0.906374
4   -0.224139
5   -0.633858
dtype: float64

In [61]:
#preenchendo de acordo com o grupo
states = ['Ohio', 'New Your', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

In [62]:
group_key = ['East'] * 4 + ['West'] * 4

In [63]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.324559
New Your      0.026962
Vermont      -0.200493
Florida      -1.449512
Oregon       -1.440504
Nevada        0.527384
California    0.007224
Idaho         1.804063
dtype: float64

In [64]:
#Definindo alguns valores como ausentes
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.324559
New Your      0.026962
Vermont            NaN
Florida      -1.449512
Oregon       -1.440504
Nevada             NaN
California    0.007224
Idaho              NaN
dtype: float64

In [65]:
#Preenchendo usando as medias dos grupos
fill_mean = lambda g: g.fillna(g.mean())

In [66]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.324559
New Your      0.026962
Vermont      -0.582370
Florida      -1.449512
Oregon       -1.440504
Nevada       -0.716640
California    0.007224
Idaho        -0.716640
dtype: float64

In [67]:
#Preencher com valores pré-definidos
fill_values = {'East': 0.5, 'West': -1}


In [68]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [69]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.324559
New Your      0.026962
Vermont       0.500000
Florida      -1.449512
Oregon       -1.440504
Nevada       -1.000000
California    0.007224
Idaho        -1.000000
dtype: float64

**Amostragem aleatória e permutação**<br>

Uma maneira de construir um baralho em inglês

In [70]:
#Copas (Hearts), Espada (Spades), Paus(Clubs), Ouro (Diamonds)
suits = ['H', 'S', 'C', 'D']
card_val=(list(range(1,11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2,11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
  cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)

In [71]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [72]:
#sortear uma mão de cinco baralhos

def draw(deck, n=5):
  return deck.sample(n)

draw(deck)

KC    10
JD    10
AC     1
5H     5
QC    10
dtype: int64

In [73]:
#Duas cartas aleatórias de cada naipe
get_suit = lambda card: card[-1] #a ultima letra é o naipe
deck.groupby(get_suit).apply(draw, n=2)

C  QC    10
   JC    10
D  5D     5
   KD    10
H  5H     5
   JH    10
S  7S     7
   5S     5
dtype: int64

In [74]:
#Outra forma de fazer a mesma coisa da célula anterior
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

KC     10
4C      4
3D      3
8D      8
5H      5
10H    10
QS     10
7S      7
dtype: int64

**Média ponderada de grupos e correlação**

In [75]:
df = pd.DataFrame({'category': ['a','a','a','a','b','b','b','b'], 'data': np.random.randn(8), 'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.657151,0.709631
1,a,1.326900,0.577720
2,a,0.793720,0.894361
3,a,1.018203,0.606043
4,b,-0.853526,0.460248
5,b,-0.081822,0.582106
6,b,-0.484096,0.844015
7,b,0.837172,0.435769


In [76]:
grouped = df.groupby('category')

In [77]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [78]:
grouped.apply(get_wavg)

category
a    0.918251
b   -0.208529
dtype: float64

In [79]:
#Preço das ações
close_px = pd.read_csv('/content/drive/MyDrive/dados_pandas/pydata-book-2nd-edition/examples/stock_px_2.csv', parse_dates=True, index_col=0)

In [80]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [81]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [82]:
#Calcula a correlação
spx_corr = lambda x: x.corrwith(x['SPX'])

In [83]:
#Calcula as mudanças percentuais
rets = close_px.pct_change().dropna()

In [84]:
#agrupar as mudanças percentuais por ano
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [85]:
#Calculo de correlação entre as colunas
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

**Regessão linear nos grupos**

In [86]:
#usando a biblioteca statsmodel

def regress(data, yvar, xvars):
  Y = data[yvar]
  X = data[xvars]
  
  X['intercept'] = 1
  result = sm.OLS(Y, X).fit()
  return result.params

In [87]:
#regressão linear de AAPL

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# Tabelas pivôs e tabulação cruzada

In [88]:
#Calcular uma tabela de médias de grupos

tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [89]:
#agrupar por tip_pct e size de acordo com time
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [90]:
#Expandir a tabela incluindo totais parciais

tips.pivot_table(['tip_pct','size'], index=['time','day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [92]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [95]:
tips.pivot_table('tip_pct', index=['time','size','smoker'], columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

**Tabulações cruzadas: crosstab**

In [98]:
data = pd.read_csv('/content/drive/MyDrive/dados_pandas/pydata-book-2nd-edition/examples/nationality.csv')

In [99]:
data

,sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left- handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left- handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left- handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [100]:
#Resumir de acordo com a nacionalidade e se são destras ou canhotas
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left- handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [101]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244